In [1]:
#!  pip install overpy  
#!  pip install routingpy

In [2]:
#! pip install gym
#! pip install tianshou
#! pip install ray

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import gym
from gym import spaces
import numpy as np
import torch
import matplotlib.pyplot as plt
from tianshou.data import Batch

C:\Users\Jatin\anaconda3\lib\site-packages\h5py\__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters
C:\Users\Jatin\anaconda3\lib\site-packages\numba\core\types\__init__.py:108: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  long_ = _make_signed(np.long)


In [5]:
import tianshou
from typing import Any, Callable, List, Optional, Tuple, Union, Dict
from tianshou.data import Batch
from tianshou.env import DummyVectorEnv
from tianshou.data import Batch, ReplayBuffer, to_torch, to_torch_as
from tianshou.policy import BasePolicy

from tianshou.env.worker import (
    DummyEnvWorker,
    EnvWorker,
    RayEnvWorker,
    SubprocEnvWorker,
)

C:\Users\Jatin\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
from env.VRPEnv import VRPEnv

In [7]:
from nets.attention_model import AttentionModel

In [8]:
# Importing required libraries

import overpy
import requests 
from bs4 import BeautifulSoup
from urllib import parse
import random
import pickle
import numpy as np
import pandas as pd

#setup OSRM 
from routingpy import OSRM
from routingpy.routers import options
options.default_timeout=None

In [9]:
import sys
from torch.utils.data import DataLoader, Dataset

C:\Users\Jatin\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# For a given address query [city_name, shop_type], first we get co-ordinates and distance values from Open Street Map
# Then generate a dataset of n_instances having graph structure information (node and edge features)

# Other shop types : supermarket, convenience, clothes, hairdresser, car_repair, bakery

def get_area_code(address):
	url = "https://www.openstreetmap.org/geocoder/search_osm_nominatim?query=" + parse.quote(address)
	r = requests.get(url) 
	soup = BeautifulSoup(r.content, 'html5lib')
	osm_link = soup.find('a', attrs = {'class':'set_position'})
	relation_id = osm_link.get('data-id').strip()	
	return int(relation_id) + 3600000000 # 3600000000 is the offset in ids 



def get_coordinates(address, shop_type):
    # Setting up 
    area_code = get_area_code(address)
    api = overpy.Overpass()

    request = api.query(f"""area({area_code});
    (node[shop={shop_type}](area);
    way[shop={shop_type}](area);
    rel[shop={shop_type}](area);
    ); out center;""")

    coords = [[float(node.lon), float(node.lat)] for node in request.nodes]
    coords += [[float(way.center_lon), float(way.center_lat)] for way in request.ways]
    coords += [[float(rel.center_lon), float(rel.center_lat)] for rel in request.relations]

    print(f"Total {len(coords)} points found on the map for search query: {address, shop_type}")
    return coords



def generate_graphs(address, shop_type, graph_size, n_instances, edge_type = "distance", out_path = "./cvrp_data/test.pk"):
    coordinates = get_coordinates(address, shop_type)
    
    client = OSRM(base_url="https://router.project-osrm.org")
    instances = []

    for _ in range (n_instances):
        instance_coords = random.sample(coordinates, graph_size) # Node coordinates for the graph are obtained by randomly sampling graph_size nodes from all queried locations on OSM
        dist_matrix = client.matrix(locations=instance_coords, profile="car")

        if edge_type == "distance":
            edge_features = np.array(dist_matrix.distances)
        elif edge_type == "time":
            edge_features = np.array(dist_matrix.durations)
            
        edge_features = torch.from_numpy(edge_features).float()
        instance = {"coordinates": instance_coords, "edge_features": edge_features}
        instances.append(instance)


    # Now generating node features for all instances -- assigning first node as depot with no demand, and all others with equal demands (for now)
    # Node features: [is_depot, is_customer, demand, coordinate_1, coordinate_2]
    all_graphs = []
    demand = 1/(len(instances[0]["coordinates"])-1) #Equal demand for all customers
    for instance in instances:
        graph_nodes = []
        coordinates = instance["coordinates"]

        for idx, node_coordinates in enumerate(coordinates):

            if idx == 0:
                node_features = np.append(np.array([1, 0, 0]), node_coordinates)
                graph_nodes = node_features
            else:
                node_features = np.append(np.array([0, 1, demand]), node_coordinates)
                graph_nodes = np.vstack((graph_nodes, node_features))
                
        graph_nodes = torch.from_numpy(graph_nodes).float()
        graph_struct = {"node_features": graph_nodes, "edge_features": instance["edge_features"], "coordinates": coordinates}
        all_graphs.append(graph_struct)

    return all_graphs

In [11]:
graphs = generate_graphs(address="Berlin", shop_type="supermarket", graph_size=10, n_instances=5)
#print(graphs[0])

Total 863 points found on the map for search query: ('Berlin', 'supermarket')


In [12]:
class VRPDataset(Dataset):  
        
    def __init__(self, graph_size=10, n_instances=5, address="Berlin", shop_type="supermarket"):
      #super().__init__()
      self.vrpdataset = generate_graphs(address, shop_type, graph_size, n_instances)
    
    def __len__(self):
      return len(self.vrpdataset)

    def __getitem__(self, index):
        data = self.vrpdataset[index]
        return data

    def sample_graphs_viz(self, n_sample=5):
        sampled_graphs = random.sample(self.vrpdataset, n_sample)
        
        # define subplot grid
        fig, axs = plt.subplots(nrows=n_sample, ncols=1, figsize=(6,20))
        plt.subplots_adjust(hspace=0.5)
        fig.suptitle(f"Sampled {n_sample} graphs from the dataset", fontsize=16, y=0.95)
        
        # loop through graphs and axes
        s_idx = 0
        labels = [f"c_{i}" for i in range(len(sampled_graphs[0]["coordinates"]))]
        labels[0] = "depot"
        for graph, ax in zip(sampled_graphs, axs.ravel()):
            longitudes = [coord[0] for coord in graph["coordinates"]]
            latitudes = [coord[1] for coord in graph["coordinates"]]
            # filter df for ticker and plot on specified axes
            ax.scatter(longitudes, latitudes)

            # chart formatting
            s_idx += 1
            ax.set_title(f"Sample {s_idx}")
            ax.set_xlabel("longitudes")
            ax.set_ylabel("latitudes")

            for i, txt in enumerate(labels):
                ax.annotate(txt, (longitudes[i], latitudes[i]))

        plt.show()

In [13]:
#dataset = VRPDataset(graph_size=10, n_instances=3)

In [14]:
env_fns = [lambda instance=graph, idx=i: VRPEnv(instance, idx) for i,graph in enumerate(graphs)]

In [15]:
test_envs = DummyVectorEnv(env_fns)

In [16]:
train_envs = DummyVectorEnv(env_fns)

In [17]:
model = AttentionModel(
        embedding_dim=64,
        hidden_dim=16,
        n_encode_layers=2,
        tanh_clipping=10.,
        mask_inner=True, #to be fixed
        mask_logits=True,
        normalization='batch',
        n_heads=8,
        checkpoint_encoder=False,
        shrink_size=None
    )

#print(model)

##print(obs['demand'][obs['ids'], :] + obs['used_capacity'][0][:, :, None] > obs['capacity'] )
#action = model.forward(obs)
#print(action)

In [18]:
class REINFORCEPolicy(BasePolicy):
    """Implementation of REINFORCE algorithm."""
    def __init__(self, model: torch.nn.Module, optim: torch.optim.Optimizer,):
        super().__init__()
        self.actor = model
        self.optim = optim
        # action distribution
        self.dist_fn = torch.distributions.Categorical

        
    def forward(self, batch: Batch) -> Batch:
        """Compute action over the given batch data."""
        logits = self.actor(batch)
        #print(f"logits: {logits}")
        dist = self.dist_fn(logits=logits)
        #print(dist.probs)
        act = self.get_actions(dist, batch)
        #print(batch["action_mask"])
        #print(batch["remaining_capacity"])
        #print(act)
        return Batch(act=act, dist=dist)


    def process_fn(self, batch: Batch, buffer: ReplayBuffer, indices: np.ndarray) -> Batch:
        """Compute the discounted returns for each transition."""
        returns, _ = self.compute_episodic_return(batch, buffer, indices, gamma=0.99, gae_lambda=1.0)
        batch.returns = returns
        print(f"pre-batch: {batch}")
        return batch


    def learn(self, batch: Batch, batch_size: int, repeat: int) -> Dict[str, List[float]]:
        """Perform the back-propagation."""
        logging_losses = []
        for _ in range(repeat):
            for minibatch in batch.split(batch_size, merge_last=True):
                self.optim.zero_grad()
                result = self(minibatch)
                dist = result.dist
                act = to_torch_as(minibatch.act, result.act)
                ret = to_torch(minibatch.returns, torch.float, result.act.device)
                log_prob = dist.log_prob(act).reshape(len(ret), -1).transpose(0, 1)
                loss = -(log_prob * ret).mean()
                loss.backward()
                self.optim.step()
                logging_losses.append(loss.item())
        return {"loss": logging_losses}
    
    
    def get_actions(self, dist, batch_obs):
        actions = []
        for idx, (probs, mask) in enumerate(zip(dist.probs, batch_obs["action_mask"])):
            demands = batch_obs["node_features"][idx,:,2]
            mask_tensor = ~torch.tensor(mask, dtype=torch.bool)
            
            action_order = np.array(torch.topk(probs, 3).indices)
            
            #print(probs)
            #print(action_order)
            
            n = int(action_order.shape[1])
            
            action = int(action_order[0,0])
            
            if action == 0: #If selected node is depot node, confirm if any step is possible to next probable node
                alt_action = int(action_order[0,1])
                if demands[alt_action] <= batch_obs["remaining_capacity"][idx]:
                    action = alt_action
                    
            if batch_obs["remaining_capacity"][idx] < demands[action]:
                action = 0
                    
            #print(action)
            actions.append(action)
        actions = torch.tensor(actions).view(-1, 1)
        return actions


In [19]:
optim = torch.optim.Adam(model.parameters(), lr=0.0003)
VRPpolicy = REINFORCEPolicy(model, optim)

C:\Users\Jatin\anaconda3\lib\site-packages\numba\np\arraymath.py:3806: DeprecationWarning: `np.MachAr` is deprecated (NumPy 1.22).
  @overload(np.MachAr)
C:\Users\Jatin\anaconda3\lib\site-packages\numba\core\ir_utils.py:1525: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if (hasattr(numpy, value)
C:\Users\Jatin\anaconda3\lib\site-packages\numba\core\ir_utils.py:1525: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: h

In [20]:
#print(VRPpolicy)
#print("========================================")
#for para in VRPpolicy.parameters():
    #print(para.shape)

C:\Users\Jatin\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
init_obs = Batch(test_envs.reset())

Env: 0 reset
Env: 1 reset
Env: 2 reset
Env: 3 reset
Env: 4 reset


In [22]:
action = VRPpolicy(init_obs)
print(action)

KeyError: 'obs'

In [ ]:
#test_envs.reset()
#result = test_envs.step(action.act)
#obs_next, rew, done, info = result
#obs_next = Batch(obs_next)
#
#print(obs_next["action_mask"])
#print(obs_next["curr_pos_idx"])

In [ ]:
test_envs.reset()
obs_next = init_obs
for _ in range(10):
    print("\n------------------------------------------------")
    action2 = VRPpolicy(obs_next)
    result2 = test_envs.step(action2.act)
    obs_next, rew, done, info = result2
    obs_next = Batch(obs_next)
    print(obs_next["action_mask"])
    #print(obs_next["curr_pos_idx"])

In [ ]:
print(obs_next)

In [ ]:
init_obs

In [ ]:
from data.VRPCollector import Collector

In [ ]:
from tianshou.data import VectorReplayBuffer
buffer_size = 100
replaybuffer = VectorReplayBuffer(buffer_size, buffer_num=5)


#test_collector = Collector(VRPpolicy, test_envs, replaybuffer)

In [ ]:
test_collector = Collector(VRPpolicy, test_envs)
train_collector = Collector(VRPpolicy, train_envs, replaybuffer)

In [ ]:
#print(replaybuffer["obs_next"])

In [ ]:
collect_result = test_collector.collect(n_episode=10)
print(collect_result)
print("Rewards of all environments are {}".format(collect_result["rews"]))
print("Average episode reward is {}.".format(collect_result["rew"]))
print("Average episode length is {}.".format(collect_result["len"]))

In [ ]:
#replaybuffer.sample(9)

#test_collector = Collector(VRPpolicy, test_envs, replaybuffer)

In [ ]:
from tianshou.trainer import onpolicy_trainer

train_collector.reset()
train_envs.reset()
test_collector.reset()
test_envs.reset()
replaybuffer.reset()

result = onpolicy_trainer(
    VRPpolicy,
    train_collector,
    test_collector,
    max_epoch=10,
    step_per_epoch=10,
    repeat_per_collect=1,
    episode_per_test=1,
    episode_per_collect=10,
    batch_size=5,
)

print(result)

In [ ]:
result